In [15]:
# Standard libraries
import os
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
from plotnine import *
from IPython.display import display

# Machine Learning
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from scipy.cluster import hierarchy as hc
from fastai.imports import *

In [16]:

data = pd.read_csv('../etc/data/lownoise/auto2.csv')
#data = pd.read_csv('../etc/data/highnoise/auto2.csv')
data.shape

(93, 23)

In [17]:
data.iloc[0,:]

maker                             Acura
 type                             Small
 CityMPG+                            25
 HighwayMPG+                         31
 Air_Bags_standard                    0
 Drive_train_type                     1
 Number_of_cylinders                  4
 Engine_size                        1.8
 Horsepower                         140
 RPM                               6300
 Engine_revolutions_per_mile       2890
 manual_transmission_available        1
 Fuel_tank_capacity                13.2
 Passenger_capacity                   5
 Length                             177
 Wheelbase                          102
 Width                               68
 U-turn_space                        37
 Rear_seat_room                    26.5
 Luggage_capacity                    11
 Weight-                           2705
 domestic                             0
 Class-                            15.9
Name: 0, dtype: object

In [18]:
def addnan(encoder , datanp, col):
    classes1 = list(encoder.classes_)
    if np.nan in classes1:
        id = classes1.index(np.nan)
        for i,value in enumerate(datanp[:,col]):
            if value == id:
                datanp[i][col] = np.nan

In [19]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import KNNImputer
imp = KNNImputer(n_neighbors=5, weights="uniform") # or ‘distance’

In [20]:
data.replace('?', np.nan, inplace=True)
datanp = data.to_numpy()
datanp[-5]

array(['Volkswagen', 'Van', 17, 21, '0', '1', '5', '2.5', '109', '4500',
       '2915', 1, '21.1', '7', '187', '115', '72', '38', '34', nan, 3960,
       0, 19.7], dtype=object)

In [21]:
from sklearn.preprocessing import LabelEncoder
encoder1 = LabelEncoder()
encod_data1 = encoder1.fit_transform(datanp[:,0])
datanp[:,0] = encod_data1
datanp

array([[0, 'Small', 25, ..., 2705, 0, 15.9],
       [0, 'Midsize', 18, ..., 3560, 0, 33.9],
       [1, 'Compact', 20, ..., 3375, 0, 29.1],
       ...,
       [29, 'Sporty', 18, ..., 2810, 0, 23.3],
       [30, 'Compact', 21, ..., 2985, 0, 22.7],
       [30, 'Midsize', 20, ..., 3245, 0, 26.7]], dtype=object)

In [22]:
encoder2 = LabelEncoder()
encod_data2 = encoder2.fit_transform(datanp[:,1])
datanp[:,1] = encod_data2
datanp

array([[0, 3, 25, ..., 2705, 0, 15.9],
       [0, 2, 18, ..., 3560, 0, 33.9],
       [1, 0, 20, ..., 3375, 0, 29.1],
       ...,
       [29, 4, 18, ..., 2810, 0, 23.3],
       [30, 0, 21, ..., 2985, 0, 22.7],
       [30, 2, 20, ..., 3245, 0, 26.7]], dtype=object)

In [23]:
dataimp = imp.fit_transform(datanp)
dataimp

array([[0.000e+00, 3.000e+00, 2.500e+01, ..., 2.705e+03, 0.000e+00,
        1.590e+01],
       [0.000e+00, 2.000e+00, 1.800e+01, ..., 3.560e+03, 0.000e+00,
        3.390e+01],
       [1.000e+00, 0.000e+00, 2.000e+01, ..., 3.375e+03, 0.000e+00,
        2.910e+01],
       ...,
       [2.900e+01, 4.000e+00, 1.800e+01, ..., 2.810e+03, 0.000e+00,
        2.330e+01],
       [3.000e+01, 0.000e+00, 2.100e+01, ..., 2.985e+03, 0.000e+00,
        2.270e+01],
       [3.000e+01, 2.000e+00, 2.000e+01, ..., 3.245e+03, 0.000e+00,
        2.670e+01]])

In [24]:
dataimp[:,0:7] = np.round(dataimp[:,0:7])
dataimp[:,8:12] = np.round(dataimp[:,8:12])
dataimp[:,13:-5] = np.round(dataimp[:,13:-5])
dataimp[:,-4:-1] = np.round(dataimp[:,-4:-1])

In [25]:
decode = encoder1.inverse_transform(dataimp[:,0].astype(np.int32))
decode1 = encoder2.inverse_transform(dataimp[:,1].astype(np.int32))
Data = np.column_stack((decode, decode1, dataimp[:,2:]))
tmp = Data[:,-2].astype(int)
newData = np.column_stack((Data[:,:-2], tmp, Data[:,-1]))
newData[-5]

array(['Volkswagen', 'Van', 17.0, 21.0, 0.0, 1.0, 5.0, 2.5, 109.0, 4500.0,
       2915.0, 1.0, 21.1, 7.0, 187.0, 115.0, 72.0, 38.0, 34.0, 16.0,
       3960.0, 0, 19.7], dtype=object)

In [26]:
dfnew = pd.DataFrame(newData, columns=data.columns)
dfnew.to_csv("knn_low_auto2.csv", index=False)
#dfnew.to_csv("knn_high_auto2.csv", index=False)